_한국어로 기계번역됨_


# 다중 에이전트 애플리케이션에서 다중 턴 대화 추가하는 방법 (기능적 API)

!!! 정보 "전제 조건"
    이 가이드는 다음에 대한 친숙성을 가정합니다:

    - [다중 에이전트 시스템](../../concepts/multi_agent)
    - [인간 개입](../../concepts/human_in_the_loop)
    - [기능적 API](../../concepts/functional_api)
    - [명령어](../../concepts/low_level/#command)
    - [LangGraph 용어집](../../concepts/low_level/)


이 방법 가이드에서는 최종 사용자가 하나 이상의 에이전트와 *다중 턴 대화*에 참여할 수 있는 애플리케이션을 구축합니다. 우리는 사용자의 입력을 수집하고 **활성** 에이전트로 다시 라우트하는 [`interrupt`](../../reference/types/#langgraph.types.interrupt)를 사용하는 노드를 만들 것입니다.

에이전트는 작업 흐름 내에서 작업으로 구현되며, 에이전트 단계를 실행하고 다음 작업을 결정합니다:

1. **대화를 계속하기 위한 사용자 입력을 기다리기**, 또는
2. **다른 에이전트로 라우트 하기** (또는 루프 내에서 다시 자신에게) [**핸드오프**](../../concepts/multi_agent/#handoffs)를 통해.

```python
from langgraph.func import entrypoint, task
from langgraph.prebuilt import create_react_agent
from langchain_core.tools import tool
from langgraph.types import interrupt


# 다른 에이전트로의 핸드오프 의사를 신호하기 위한 도구 정의
# 참고: 이는 다른 에이전트로 이동하기 위한 Command(goto) 구문을 사용하지 않음:
# 아래의 `workflow()`가 핸드오프를 명시적으로 처리합니다.
@tool(return_direct=True)
def transfer_to_hotel_advisor():
    """호텔 상담 에이전트에게 도움 요청하기."""
    return "호텔 상담 에이전트로 성공적으로 전환되었습니다."


# 에이전트 정의
travel_advisor_tools = [transfer_to_hotel_advisor, ...]
travel_advisor = create_react_agent(model, travel_advisor_tools)


# 에이전트를 호출하는 작업 정의
@task
def call_travel_advisor(messages):
    response = travel_advisor.invoke({"messages": messages})
    return response["messages"]


# 다중 에이전트 네트워크 작업 흐름 정의
@entrypoint(checkpointer)
def workflow(messages):
    call_active_agent = call_travel_advisor
    while True:
        agent_messages = call_active_agent(messages).result()
        ai_msg = get_last_ai_msg(agent_messages)
        if not ai_msg.tool_calls:
            user_input = interrupt(value="사용자 입력을 받을 준비가 되었습니다.")
            messages = messages + [{"role": "user", "content": user_input}]
            continue

        messages = messages + agent_messages
        call_active_agent = get_next_agent(messages)
    return entrypoint.final(value=agent_messages[-1], save=messages)
```


## 설치

먼저 필요한 패키지를 설치합시다.


In [1]:
# %%capture --no-stderr
# %pip install -U langgraph langchain-anthropic


In [2]:
import getpass
import os


def _set_env(var: str):
    if not os.environ.get(var):
        os.environ[var] = getpass.getpass(f"{var}: ")


_set_env("ANTHROPIC_API_KEY")


ANTHROPIC_API_KEY:  ········


<div class="admonition tip">
    <p class="admonition-title">LangGraph 개발을 위한 <a href="https://smith.langchain.com">LangSmith</a> 설정하기</p>
    <p style="padding-top: 5px;">
        LangSmith에 가입하여 LangGraph 프로젝트의 문제를 신속하게 파악하고 성능을 개선하세요. LangSmith는 트레이스 데이터를 사용하여 LangGraph로 구축된 LLM 앱을 디버그하고, 테스트하며, 모니터링할 수 있도록 해줍니다 — 시작하는 방법에 대해 더 읽으려면 <a href="https://docs.smith.langchain.com">여기</a>를 클릭하세요.
    </p>
</div>


이 예제에서는 서로 소통할 수 있는 여행 어시스턴트 에이전트 팀을 구축합니다.

2명의 에이전트를 만들 것입니다:

* `여행_조언자`: 여행 목적지 추천을 도와줄 수 있습니다. `호텔_조언자`에게 도움을 요청할 수 있습니다.
* `호텔_조언자`: 호텔 추천을 도와줄 수 있습니다. `여행_조언자`에게 도움을 요청할 수 있습니다.

이는 완전히 연결된 네트워크입니다 - 모든 에이전트는 다른 모든 에이전트와 대화할 수 있습니다.


In [3]:
import random
from typing_extensions import Literal
from langchain_core.tools import tool


@tool
def get_travel_recommendations():
    """Get recommendation for travel destinations"""
    return random.choice(["aruba", "turks and caicos"])


@tool
def get_hotel_recommendations(location: Literal["aruba", "turks and caicos"]):
    """Get hotel recommendations for a given destination."""
    return {
        "aruba": [
            "The Ritz-Carlton, Aruba (Palm Beach)"
            "Bucuti & Tara Beach Resort (Eagle Beach)"
        ],
        "turks and caicos": ["Grace Bay Club", "COMO Parrot Cay"],
    }[location]


@tool(return_direct=True)
def transfer_to_hotel_advisor():
    """Ask hotel advisor agent for help."""
    return "Successfully transferred to hotel advisor"


@tool(return_direct=True)
def transfer_to_travel_advisor():
    """Ask travel advisor agent for help."""
    return "Successfully transferred to travel advisor"


!!! 참고 "전송 도구"

    전송 도구에서 `@tool(return_direct=True)`를 사용하는 것을 아마 눈치채셨을 것입니다. 이는 개별 에이전트(예: `travel_advisor`)가 이 도구가 호출되면 ReAct 루프에서 조기에 종료할 수 있도록 하기 위함입니다. 이는 우리가 에이전트가 이 도구를 호출할 때를 감지하고 즉시 다른 에이전트에게 제어를 넘기기를 원하기 때문에 바람직한 동작입니다.
    
    **참고**: 이는 미리 구축된 [`create_react_agent`][langgraph.prebuilt.chat_agent_executor.create_react_agent]와 함께 작동하도록 설계되었습니다. 사용자 정의 에이전트를 구축하는 경우 `return_direct`로 표시된 도구에 대한 조기 종료 처리를 위한 논리를 수동으로 추가해야 합니다.


이제 미리 만들어진 [`create_react_agent`][langgraph.prebuilt.chat_agent_executor.create_react_agent]와 우리의 다중 에이전트 워크플로를 사용하여 에이전트를 생성해 보겠습니다. 각 에이전트로부터 최종 응답을 받은 후마다 [`interrupt`][langgraph.types.interrupt]를 호출할 것임을 유의하세요.


In [6]:
import uuid

from langchain_core.messages import AIMessage
from langchain_anthropic import ChatAnthropic
from langgraph.prebuilt import create_react_agent
from langgraph.graph import add_messages
from langgraph.func import entrypoint, task
from langgraph.checkpoint.memory import MemorySaver
from langgraph.types import interrupt, Command

model = ChatAnthropic(model="claude-3-5-sonnet-latest")

# Define travel advisor ReAct agent
travel_advisor_tools = [
    get_travel_recommendations,
    transfer_to_hotel_advisor,
]
travel_advisor = create_react_agent(
    model,
    travel_advisor_tools,
    state_modifier=(
        "You are a general travel expert that can recommend travel destinations (e.g. countries, cities, etc). "
        "If you need hotel recommendations, ask 'hotel_advisor' for help. "
        "You MUST include human-readable response before transferring to another agent."
    ),
)


@task
def call_travel_advisor(messages):
    # You can also add additional logic like changing the input to the agent / output from the agent, etc.
    # NOTE: we're invoking the ReAct agent with the full history of messages in the state
    response = travel_advisor.invoke({"messages": messages})
    return response["messages"]


# Define hotel advisor ReAct agent
hotel_advisor_tools = [get_hotel_recommendations, transfer_to_travel_advisor]
hotel_advisor = create_react_agent(
    model,
    hotel_advisor_tools,
    state_modifier=(
        "You are a hotel expert that can provide hotel recommendations for a given destination. "
        "If you need help picking travel destinations, ask 'travel_advisor' for help."
        "You MUST include human-readable response before transferring to another agent."
    ),
)


@task
def call_hotel_advisor(messages):
    response = hotel_advisor.invoke({"messages": messages})
    return response["messages"]


checkpointer = MemorySaver()


def string_to_uuid(input_string):
    return str(uuid.uuid5(uuid.NAMESPACE_URL, input_string))


@entrypoint(checkpointer=checkpointer)
def multi_turn_graph(messages, previous):
    previous = previous or []
    messages = add_messages(previous, messages)
    call_active_agent = call_travel_advisor
    while True:
        agent_messages = call_active_agent(messages).result()
        messages = add_messages(messages, agent_messages)
        # Find the last AI message
        # If one of the handoff tools is called, the last message returned
        # by the agent will be a ToolMessage because we set them to have
        # "return_direct=True". This means that the last AIMessage will
        # have tool calls.
        # Otherwise, the last returned message will be an AIMessage with
        # no tool calls, which means we are ready for new input.
        ai_msg = next(m for m in reversed(agent_messages) if isinstance(m, AIMessage))
        if not ai_msg.tool_calls:
            user_input = interrupt(value="Ready for user input.")
            # Add user input as a human message
            # NOTE: we generate unique ID for the human message based on its content
            # it's important, since on subsequent invocations previous user input (interrupt) values
            # will be looked up again and we will attempt to add them again here
            # `add_messages` deduplicates messages based on the ID, ensuring correct message history
            human_message = {
                "role": "user",
                "content": user_input,
                "id": string_to_uuid(user_input),
            }
            messages = add_messages(messages, [human_message])
            continue

        tool_call = ai_msg.tool_calls[-1]
        if tool_call["name"] == "transfer_to_hotel_advisor":
            call_active_agent = call_hotel_advisor
        elif tool_call["name"] == "transfer_to_travel_advisor":
            call_active_agent = call_travel_advisor
        else:
            raise ValueError(f"Expected transfer tool, got '{tool_call['name']}'")

    return entrypoint.final(value=agent_messages[-1], save=messages)


## 다중 대화 테스트

이 애플리케이션으로 다중 대화를 테스트해 봅시다.


In [7]:
thread_config = {"configurable": {"thread_id": uuid.uuid4()}}

inputs = [
    # 1st round of conversation,
    {
        "role": "user",
        "content": "i wanna go somewhere warm in the caribbean",
        "id": str(uuid.uuid4()),
    },
    # Since we're using `interrupt`, we'll need to resume using the Command primitive.
    # 2nd round of conversation,
    Command(
        resume="could you recommend a nice hotel in one of the areas and tell me which area it is."
    ),
    # 3rd round of conversation,
    Command(
        resume="i like the first one. could you recommend something to do near the hotel?"
    ),
]

for idx, user_input in enumerate(inputs):
    print()
    print(f"--- Conversation Turn {idx + 1} ---")
    print()
    print(f"User: {user_input}")
    print()
    for update in multi_turn_graph.stream(
        user_input,
        config=thread_config,
        stream_mode="updates",
    ):
        for node_id, value in update.items():
            if isinstance(value, list) and value:
                last_message = value[-1]
                if isinstance(last_message, dict) or last_message.type != "ai":
                    continue
                print(f"{node_id}: {last_message.content}")



--- Conversation Turn 1 ---

User: {'role': 'user', 'content': 'i wanna go somewhere warm in the caribbean', 'id': 'f48d82a7-7efa-43f5-ad4c-541758c95f61'}

call_travel_advisor: Based on the recommendations, Aruba would be an excellent choice for your Caribbean getaway! Known as "One Happy Island," Aruba offers:
- Year-round warm weather with consistent temperatures around 82°F (28°C)
- Beautiful white sand beaches like Eagle Beach and Palm Beach
- Crystal clear waters perfect for swimming and snorkeling
- Minimal rainfall and location outside the hurricane belt
- Rich culture blending Dutch and Caribbean influences
- Various activities from water sports to desert-like landscape exploration
- Excellent dining and shopping options

Would you like me to help you find suitable accommodations in Aruba? I can transfer you to our hotel advisor who can recommend specific hotels based on your preferences.

--- Conversation Turn 2 ---

User: Command(resume='could you recommend a nice hotel in o